In [53]:
import pandas as pd

# Read x_train.txt
X = pd.read_csv('../data/x_train.txt', sep=' ', header=None)

# Read y_train.txt
y = pd.read_csv('../data/y_train.txt', sep=' ', header=None)

X_real = pd.read_csv('../data/x_test.txt', sep=' ', header=None)


# Display the data
print("x:")
print(X.head())

print("\ny:")
print(y.head())

# Display the data
print("x:")
print(X_real.head())

x:
        0         1         2         3         4         5         6    \
0 -2.619773 -2.619533 -1.199350 -1.083335 -1.000910 -0.366967 -2.164037   
1 -1.415579 -1.782544 -2.880270 -1.958863  1.159968  0.273030 -1.628728   
2 -2.745092 -1.382945 -1.626015 -1.282560 -0.663146  0.052349 -2.403322   
3  0.618998  0.455364 -0.115081  0.649040 -0.862207  2.308504  0.526114   
4 -0.070694 -0.550509 -0.565556 -0.693065 -0.573089 -0.395862  0.003170   

        7         8         9    ...        490        491        492  \
0 -1.210001 -0.658311 -1.489539  ...  10.849925  10.343346  10.717519   
1 -0.175813 -0.916857 -0.570166  ...  11.489417   5.195818   3.494627   
2 -0.765073 -0.394354 -0.806624  ...  13.934934   9.267515   4.705604   
3 -1.094852  1.088656 -0.481210  ...  12.021328   3.852231  11.059702   
4 -0.981609 -0.505775 -0.758430  ...   7.537788  11.229665  11.318915   

        493        494        495        496        497        498        499  
0  7.709295   5.894554  12.

## generation for NM + [102, 103, 105]

In [60]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

cols=[102, 103, 105]
Xl = X[cols]
X_real_nb = X_real[cols]

model = GaussianNB()
model.fit(Xl, y)

preds_proba = model.predict_proba(X_real_nb)[:, 1]

top_20_percent_indices = np.argsort(preds_proba)[-int(0.2 * len(preds_proba)):]
#print(top_20_percent_indices)
#print(top_20_percent_indices+1)
# Increment each index by one
top_20_percent_indices += 1

# Save to a file
np.savetxt("result-nb.txt", top_20_percent_indices, fmt='%d')



d:\mgr\aml\pr2\venv\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##  generation for ensemble4 + [102, 103, 105]

In [61]:
import ast
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

df_e4 = pd.read_csv('ensemble4-raytune-bohb.csv')
config_row = df_e4.loc[df_e4['custom_score'].idxmax()]
config = {
    "colset": config_row['config/colset'],
    "lr_C": config_row['config/lr_C'],
    "lr_solver": config_row['config/lr_solver'],
    "svc_C": config_row['config/svc_C'],
    "svc_kernel": config_row['config/svc_kernel'],
    "weight_nb": config_row['config/weight_nb'],
    "weight_lr": config_row['config/weight_lr'],
    "weight_svc": config_row['config/weight_svc'],
    "weight_xgb": config_row['config/weight_xgb'],
    "xgb_alpha": config_row['config/xgb_alpha'],
    "xgb_colsample_bytree": config_row['config/xgb_colsample_bytree'],
    "xgb_eta": config_row['config/xgb_eta'],
    "xgb_lambda": config_row['config/xgb_lambda'],
    "xgb_max_depth": config_row['config/xgb_max_depth'],
    "xgb_min_child_weight": config_row['config/xgb_min_child_weight'],
    "xgb_subsample": config_row['config/xgb_subsample']
}

models = [
        ('xgb', XGBClassifier(
            eta=config["xgb_eta"],
            max_depth=config["xgb_max_depth"],
            min_child_weight=config["xgb_min_child_weight"],
            subsample=config["xgb_subsample"],
            colsample_bytree=config["xgb_colsample_bytree"],
            reg_lambda=config["xgb_lambda"],
            reg_alpha=config["xgb_alpha"],
            objective="binary:logistic",
            eval_metric="logloss",
            use_label_encoder=False,
            verbosity=0
        )),
        ('lr', LogisticRegression(
            C=config["lr_C"],
            solver=config["lr_solver"],
            max_iter=1000
        )),            
        ('svc', SVC(
            C=config["svc_C"],
            kernel=config["svc_kernel"],
            probability=True
        )),
        ('nb', GaussianNB()),
    ]
weights = [
    config["weight_xgb"],
    config["weight_lr"],
    config["weight_svc"],
    config["weight_nb"]
]
model = VotingClassifier(estimators=models, voting='soft', weights=weights)

vars = ast.literal_eval(config['colset'])
Xloc = X[vars]
X_real_e = X_real[vars]

model.fit(Xloc, y)
preds_proba = model.predict_proba(X_real_e)[:, 1]
top_20_percent_indices = np.argsort(preds_proba)[-int(0.2 * len(preds_proba)):]
top_20_percent_indices += 1
np.savetxt("result-enseble.txt", top_20_percent_indices, fmt='%d')

d:\mgr\aml\pr2\venv\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\mgr\aml\pr2\venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [62]:
print(vars)

[102, 103, 105]
